Observing the weights of the pre-trained models (sender and receiver)

In [2]:
import torch

state_dict = torch.load("example/receiver_weights500.pth", map_location=torch.device('cpu'))
for key in state_dict:
    print(key)


hidden_to_output.weight
hidden_to_output.bias
encoder.cell.weight_ih_l0
encoder.cell.weight_hh_l0
encoder.cell.bias_ih_l0
encoder.cell.bias_hh_l0
encoder.embedding.weight


In [3]:
import torch

state_dict = torch.load("example/sender_weights500.pth", map_location=torch.device('cpu'))
for key in state_dict:
    print(key)

sos_embedding
agent.fc1.weight
agent.fc1.bias
hidden_to_output.weight
hidden_to_output.bias
embedding.weight
cells.0.weight_ih
cells.0.weight_hh
cells.0.bias_ih
cells.0.bias_hh


Adding 10 output channels so the model can guess not only 100 input messages but 110. 

In [4]:
import torch

# Load the model weights, specifying the CPU as the target device
state_dict = torch.load("example/receiver_weights500.pth", map_location=torch.device('cpu'))

# Assume the input layer weights are stored under 'input_to_hidden.weight' 
# Adjust this name based on the model structure:
old_input_weight = state_dict['hidden_to_output.weight']  

# Create a new weight matrix with the updated input size (110)
# Assume hidden size is 250 -  adjust this if different:
new_input_weight = torch.zeros(110, 600)  

# Copy the old weights for the first 100 features
new_input_weight[:100, :] = old_input_weight

# Initialize weights for the new 10 features (e.g., randomly or with zeros)
# Here, we initialize them randomly using a normal distribution:
new_input_weight[100:, :] = torch.randn(10, 600) * 0.01  

# Update the state_dict with the new input layer weights
state_dict['hidden_to_output.weight'] = new_input_weight

# Load the model weights, specifying the CPU as the target device
#state_dict = torch.load("drive/MyDrive/Lazimpa/example/receiver_weights500.pth", map_location=torch.device('cpu'))

# Assume the input layer weights are stored under 'input_to_hidden.weight' 
# Adjust this name based on the model structure:
old_input_weight = state_dict['hidden_to_output.bias']  

# Create a new weight matrix with the updated input size (110)
# Assume hidden size is 250 -  adjust this if different:
new_input_weight = torch.zeros(110) # Assuming the output size remains the same

# Copy the old weights for the first 100 features
new_input_weight[:100] = old_input_weight # Adapting for bias (1D)

# Initialize weights for the new 10 features (e.g., randomly or with zeros)
# Here, we initialize them randomly using a normal distribution:
new_input_weight[100:] = torch.randn(10) * 0.01 # Adapting for bias (1D) 

# Update the state_dict with the new input layer weights
state_dict['hidden_to_output.bias'] = new_input_weight

# Save the modified state_dict
torch.save(state_dict, "example/receiver_weights500_updated.pth")

In [5]:
import torch

# Load the model weights, specifying the CPU as the target device
state_dict = torch.load("example/sender_weights500.pth", map_location=torch.device('cpu'))

# Assume the input layer weights are stored under 'input_to_hidden.weight' 
# Adjust this name based on the model structure:
old_input_weight = state_dict['agent.fc1.weight']  

# Create a new weight matrix with the updated input size (110)
# Assume hidden size is 250 -  adjust this if different:
new_input_weight = torch.zeros(250, 110)  

# Copy the old weights for the first 100 features
new_input_weight[:, :100] = old_input_weight

# Initialize weights for the new 10 features (e.g., randomly or with zeros)
# Here, we initialize them randomly using a normal distribution:
new_input_weight[:, 100:] = torch.randn(250, 10) * 0.01  

# Update the state_dict with the new input layer weights
state_dict['agent.fc1.weight'] = new_input_weight

# Update receiver_embedding if necessary
#state_dict['receiver_embedding'] = torch.nn.Embedding(40, 110) # Replace 40 and 110 with the actual vocab_size and embedding_dim

# Save the modified state_dict
torch.save(state_dict, "example/sender_weights500_updated.pth")

Testing the old model trained on 100 messages now on 110 messages (of course it does not work because it has not yet been fine-tuned on the 10 added messages)

In [7]:
! python3 -m egg.zoo.channel.test \
        --impatient=True \
        --save_dir="analysis/example" \
         --receiver_weights="example/receiver_weights500_updated.pth" \
         --sender_weights="example/sender_weights500_updated.pth" \
         --vocab_size=40  \
         --max_len=30 \
         --n_features=110 \
         --sender_cell="lstm" \
         --receiver_cell="lstm" \
         --sender_hidden=250 \
         --receiver_hidden=600 \
          --receiver_embedding=100 \
          --sender_embedding=10 \
          --sender_num_layers=1 \
          --receiver_num_layers=1

Namespace(n_features=110, batches_per_epoch=1000, dim_dataset=10240, force_eos=0, sender_hidden=250, receiver_hidden=600, receiver_num_layers=1, sender_num_layers=1, receiver_num_heads=8, sender_num_heads=8, sender_embedding=10, receiver_embedding=100, causal_sender=False, causal_receiver=False, sender_generate_style='in-place', sender_cell='lstm', receiver_cell='lstm', sender_entropy_coeff=0.1, receiver_entropy_coeff=0.1, probs='uniform', length_cost=0.0, name='model', early_stopping_thr=0.9999, receiver_weights='example/receiver_weights500_updated.pth', sender_weights='example/sender_weights500_updated.pth', save_dir='analysis/example', impatient=True, unigram_pen=0.0, random_seed=740693356, checkpoint_dir=None, preemptable=False, checkpoint_freq=0, validation_freq=1, n_epochs=10, load_from_checkpoint=None, no_cuda=True, batch_size=32, optimizer='adam', lr=0.01, vocab_size=40, max_len=30, tensorboard=False, tensorboard_dir='runs/', cuda=False, device='cpu')
Impatient score=220
input:

Here we must finetune the model so that it outputs the right messages for the last 10 added messages. 